In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
from os import path
from datetime import datetime, date, timedelta

from mock_shiny_inputs import Input

input = Input()

In [2]:
df_fondsen = pd.read_csv("fondsen.csv")
df_fondsen = df_fondsen.dropna()
df_fondsen = df_fondsen.set_index("id")
df_fondsen

,file_name,directory,name
id,,,
0P0000AA58,0P0000AA58 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
0P0000AA59,0P0000AA59 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
0P0000AA5A,0P0000AA5A Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
0P0000XZ4E,0P0000XZ4E Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Core Fund Bala...
0P0000XZ4F,0P0000XZ4F Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Core Fund Bala...
LP65104373,LP65104373 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
LP65104374,LP65104374 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
LP65104375,LP65104375 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...
0P00000N68,0P00000N68 Historical Data.csv,DPam,Dpam B - Balanced Growth A


## Functies

In [ ]:
def df_mnd_stortingen(df):
    storting_datums = df[df["Datum"].dt.day >= input.in_dag_storting()].groupby("Maand")["Datum"].min()
    storting_datums

    df_storting_datums = df[df["Datum"].isin(storting_datums)].copy()
    df_storting_datums["Aandelen"] = input.in_bedrag_storting() / df_storting_datums["Koers"]

    return df_storting_datums

In [ ]:
def df_mnd_verkoop_datums(df):
    verkoop_datums = df[df["Datum"].dt.day >= input.in_dag_verkoop()].groupby("Maand")["Datum"].min()

    df_verkoop_dt = df[df["Datum"].isin(verkoop_datums)].copy()

    return df_verkoop_dt

In [ ]:
from calc import *

In [ ]:
for f_id in df_fondsen.index[0:1]:
    file_name = df_fondsen.loc[f_id,"file_name"]
    directory = df_fondsen.loc[f_id,"directory"]
    name = df_fondsen.loc[f_id,"name"]
    
    save_path = path.join("Pre", f_id + ".csv")
    
    file_path = path.join("Data", directory, file_name)
    
    if path.exists(save_path):
        continue
        
    df = pd.read_csv(file_path, parse_dates=["Date"])
    df = df.rename(columns={"Date": "Datum", "Price": "Koers"})
    df = df[["Datum","Koers"]]
    if df["Koers"].dtype == "object":
        df["Koers"] = df["Koers"].str.replace(",","").astype(float)
    df["Maand"] = df["Datum"].apply(lambda x: x.replace(day=1))
    
    print(df.head())
    df_stortingen = df_mnd_stortingen(df)
    
    df_verkoop = df_mnd_verkoop_datums(df)
    df_stortingen_verkoop = pd.merge(df_stortingen["Datum"], df_verkoop["Datum"], how="cross", suffixes=("_Eerste_Storting","_Verkoop"))
    df_stortingen_verkoop = df_stortingen_verkoop[df_stortingen_verkoop["Datum_Verkoop"] > df_stortingen_verkoop["Datum_Eerste_Storting"]]
    
    #df_stortingen_verkoop[["Investering Totaal","Verkoop Totaal","Winst Pct", "Eff Interest"]] = df_stortingen_verkoop.apply(lambda x: ji(df,
    df_stortingen_verkoop.apply(lambda x: ji(df,
        df_stortingen,
        x["Datum_Eerste_Storting"],
        x["Datum_Verkoop"],
        input.in_instapkost_pct(),
        input.in_verkoop_sper_periode(),
        input.in_min_stortingen_voor_verkoop()), axis=1, result_type="expand")
    
    print(df_stortingen_verkoop)
    print("Saving to {}".format(save_path))
    df_stortingen_verkoop.to_csv(save_path)

## Display

In [ ]:
dfd = pd.read_csv("Pre\\0P0000AA58.csv", parse_dates=["Datum_Eerste_Storting","Datum_Verkoop"])
dfd
dfd = dfd[np.abs(dfd["Eff Interest"]) <= 15]
dfd

In [ ]:
n_dt_stortingen = int(dfd["Datum_Eerste_Storting"].drop_duplicates().count())
n_dt_verkoop = int(dfd["Datum_Verkoop"].drop_duplicates().count())
n_dt_stortingen, n_dt_verkoop

In [ ]:
fig = px.density_heatmap(dfd, 
                         x="Datum_Eerste_Storting", 
                         y="Datum_Verkoop", 
                         z="Eff Interest", 
                         nbinsx=n_dt_stortingen,
                         nbinsy=n_dt_verkoop,
                         histfunc="avg",
                         color_continuous_midpoint=0,
                         height=1000
                         )
fig.update_traces(dict(colorscale=["red","white","lime"], showscale=True, coloraxis=None))
fig

In [ ]:
dfd["Jaar Eerste Storting"] = dfd["Datum_Eerste_Storting"].dt.year
dfd["Jaar Verkoop"] = dfd["Datum_Verkoop"].dt.year

In [ ]:
min_year_st = dfd["Jaar Eerste Storting"].min()
max_year_st = dfd["Jaar Eerste Storting"].max()
min_year_v = dfd["Jaar Verkoop"].min()
max_year_v = dfd["Jaar Verkoop"].max()

year_list = [dfd["Eff Interest"][dfd["Jaar Eerste Storting"] == jaar] for jaar in range(min_year_st, max_year_st+1)]

In [ ]:
fig = ff.create_distplot(year_list, range(min_year, max_year+1), show_hist=False)
fig.update_layout(height=800)
fig

In [ ]:
from plotly.subplots import make_subplots
from random import choices

In [ ]:
max_year_v-min_year_v+1, max_year_st-min_year_st+1
min_year_v, max_year_v, min_year_st, max_year_st

In [ ]:
yr_st = 2018
yr_v = 2019
df_st_v = dfd["Eff Interest"][(dfd["Jaar Eerste Storting"] == yr_st) & (dfd["Jaar Verkoop"] == yr_v)]
ff.create_distplot([df_st_v], range(2018,2019), show_hist=False)


In [ ]:
years_v = range(min_year_v, max_year_v + 1)
i_v = range(max_year_v - min_year_v + 1)

years_st = range(min_year_st, max_year_st + 1)
i_st = range(max_year_st - min_year_st + 1)

rows = len(years_v)
cols = len(years_st)

print(years_v, years_st, i_v, i_st)
    
titles = [["" for i in range(cols)] for j in range(rows)]
figs = [[None for i in range(cols)] for j in range(rows)]

for r in i_v:
    yr_v = years_v[r]
    
    for c in i_st:
        #print("row {}, col {}".format(r, c))
        yr_st = years_st[c]
        #print("yr_st {}, yr_v {}".format(yr_st, yr_v))
        if yr_st <= yr_v:
        
            df_st_v = dfd["Eff Interest"][(dfd["Jaar Eerste Storting"] == yr_st) & (dfd["Jaar Verkoop"] == yr_v)]

            distplot = ff.create_distplot([df_st_v], ["Fund"], show_hist=False, show_rug=False)
            distplot.update_layout(title="({},{})".format(yr_st, yr_v))

            figs[r][c] = distplot.data[0]
            titles[r][c] = "{}-{}".format(yr_st, yr_v)
        else:
            figs[r][c] = None
            titles[r][c] = ""

fig = make_subplots(rows=rows, cols=cols, subplot_titles=[titles[r][c] for r in reversed(range(rows)) for c in range(cols)])

for r in i_v:
    for c in i_st:
        if figs[r][c]:
            #print("{}, {}".format(r, c))
            fig.add_trace(figs[r][c], row=rows-r, col=c+1)


fig.update_layout(height=1600, showlegend=False)
fig

In [ ]:
len(years_st)

### Vergelijking

In [ ]:
df_all = None

for fonds in df_fondsen.index:
    dff = pd.read_csv("Pre\\{}_3.csv".format(fonds), parse_dates=["Datum_Eerste_Storting","Datum_Verkoop"])
    dff["Fonds"] = fonds
    dff["Fonds Naam"] = df_fondsen["name"].loc[fonds]
    
    if df_all is None:
        df_all = dff
    else:
        df_all = pd.concat([df_all, dff])
      
df_all["Jaar Eerste Storting"] = df_all["Datum_Eerste_Storting"].dt.year
df_all["Jaar Verkoop"] = df_all["Datum_Verkoop"].dt.year
df_all

In [ ]:
fondsen = ["0P0000AA58","0P0000AA59"]
years_v = range(min_year_v, max_year_v + 1)
i_v = range(max_year_v - min_year_v + 1)

years_st = range(min_year_st, max_year_st + 1)
i_st = range(max_year_st - min_year_st + 1)

rows = len(years_v)
cols = len(years_st)

print(years_v, years_st, i_v, i_st)
    
titles = [["" for i in range(cols)] for j in range(rows)]
figs = [[None for i in range(cols)] for j in range(rows)]

for r in i_v:
    yr_v = years_v[r]
    
    for c in i_st:
        #print("row {}, col {}".format(r, c))
        yr_st = years_st[c]
        #print("yr_st {}, yr_v {}".format(yr_st, yr_v))
        if yr_st <= yr_v:
        
            df_st_v = [df_all["Eff Interest"][(df_all["Jaar Eerste Storting"] == yr_st) & 
                                           (df_all["Jaar Verkoop"] == yr_v) &
                                           (df_all["Fonds"] == fonds)].dropna()  for fonds in fondsen]

            max_len = max([len(df_st_v[i].index) for i in [0,1]])
            data = [0 for i in fondsen]
            
            for i in range(len(data)):
                if len(df_st_v[i]) < max_len:
                    data[i] = choices(df_st_v[i].tolist(), k=max_len)
                else:
                    data[i] = df_st_v[i].tolist()
            
            distplot = ff.create_distplot(data, fondsen, show_hist=False, show_rug=False)
            distplot.update_layout(title="({},{})".format(yr_st, yr_v))

            figs[r][c] = distplot
            titles[r][c] = "{}-{}".format(yr_st, yr_v)
        else:
            figs[r][c] = None
            titles[r][c] = ""

fig = make_subplots(rows=rows, cols=cols, subplot_titles=[titles[r][c] for r in reversed(range(rows)) for c in range(cols)])

for r in i_v:
    for c in i_st:
        if figs[r][c]:
            #print("{}, {}".format(r, c))
            for i in range(len(fondsen)):
                fig.add_trace(go.Scatter(figs[r][c].data[i], showlegend=(r+c==0)), row=rows-r, col=c+1)


fig.update_layout(
    height=1600, 
    legend=dict(
        yanchor="bottom",
        y=0.01,
        xanchor="left",
        x=0.5
    )
)

fig

In [ ]:
figs[0][0].data

In [ ]:
a = [[2.8828125, 3.8359375, 2.7890625, 2.015625, -0.5859375, -6.1796875, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-4.90625, -3.0078125, 2.78125, -3.375, 0.4765625, -3.30078125, -4.14453125, -1.34375, -4.8984375, -15.0546875, -4.796875, -2.6015625, -5.8046875, -14.46875, -20.7734375, -4.890625, -2.94140625, -5.640625, -12.9765625, -17.88671875, -14.4453125, -4.5703125, -2.69921875, -4.90625, -11.1640625, -15.04296875, -11.2109375, -6.3203125, -4.2109375, -2.39453125, -4.2421875, -9.7421875, -13.01171875, -9.2734375, -5.0078125, -10.515625, -3.81640625, -2.05078125, -3.6171875, -8.51953125, -11.33203125, -7.73046875, -3.83984375, -8.484375, -7.4140625]]

In [ ]:
ff.create_distplot(data, ["A","B"], show_hist=False, show_rug=False)

In [ ]:
b = [1,2,3]
for i in range(len(b)):
    a[i] = b[i] 

a

In [ ]:
a[0:3] = b

In [ ]:
a

In [ ]:
np.NAN

In [ ]:
df_all

In [5]:
dft = df_fondsen.copy().reset_index()
dft["id2"] = dft["id"].shift(1)
dft

,id,file_name,directory,name,id2
0,0P0000AA58,0P0000AA58 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,None
1,0P0000AA59,0P0000AA59 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,0P0000AA58
2,0P0000AA5A,0P0000AA5A Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,0P0000AA59
3,0P0000XZ4E,0P0000XZ4E Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Core Fund Bala...,0P0000AA5A
4,0P0000XZ4F,0P0000XZ4F Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Core Fund Bala...,0P0000XZ4E
5,LP65104373,LP65104373 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,0P0000XZ4F
6,LP65104374,LP65104374 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,LP65104373
7,LP65104375,LP65104375 Historical Data.csv,Ing,Ing (b) Collect Portfolio - Ing Personal Portf...,LP65104374
8,0P00000N68,0P00000N68 Historical Data.csv,DPam,Dpam B - Balanced Growth A,LP65104375
9,0P00000N6E,0P00000N6E Historical Data.csv,DPam,Dpam B - Balanced Growth B,0P00000N68
